# 1. Understanding the Dataset and Task
The HERDPhobia dataset is organized with three splits:

- Training set: Used to train the model
- Development (dev) set: Used for validation during training and hyperparameter tuning
- Test set: Used only for final evaluation

The HERDPhobia dataset is specifically about:

Detecting hate speech against Fulani herdsmen in Nigeria
Classification into 3 categories:

- Hate speech (HT)
- Non-hate speech (NHT)
- Indeterminate (IND)

# 2. Environment Setup and Data Preparation

In [ ]:
# Install required libraries
# !pip install transformers datasets pandas numpy scikit-learn torch

# Clone the repository
#!git clone https://github.com/hausanlp/HERDPhobia

In [ ]:
# Import of important library
import pandas as pd
import torch
import os
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from huggingface_hub import notebook_login
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-04-04 12:49:25.773272: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 12:49:25.875575: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Now, let's load and explore all three splits:

In [ ]:
# Load the dataset
dataset_path = "HERDPhobia/data"
train_path = os.path.join(dataset_path, "train.tsv")
dev_path = os.path.join(dataset_path, "dev.tsv")
test_path = os.path.join(dataset_path, "test.tsv")

# Read the data
train_df = pd.read_csv(train_path, sep='\t')
dev_df = pd.read_csv(dev_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')

# Explore the data
print(f"Training set size: {len(train_df)}")
print(f"Development set size: {len(dev_df)}")
print(f"Test set size: {len(test_df)}")

# Check class distribution in all splits
print("\nClass distribution in training set:")
print(train_df['label'].value_counts())
print("\nClass distribution in dev set:")
print(dev_df['label'].value_counts())
print("\nClass distribution in test set:")
print(test_df['label'].value_counts())

# Check for label encoding (if labels are categorical or numerical)
print("\nLabel values:", train_df['label'].unique())

If the labels are text-based (HT, NHT, IND), we'll need to convert them to integers

In [ ]:
# Create a label mapping if necessary
label_map = {'HT': 0, 'NHT': 1, 'IND': 2}  # Adjust based on actual label representation

# Convert text labels to integers if needed
if train_df['label'].dtype == 'object':
    train_df['label'] = train_df['label'].map(label_map)
    dev_df['label'] = dev_df['label'].map(label_map)
    test_df['label'] = test_df['label'].map(label_map)

Convert the dataset to a format suitable for Hugging Face's datasets library

In [ ]:
# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict with all three splits
herdphobia_dataset = DatasetDict({
    'train': train_dataset,
    'validation': dev_dataset,  # Using 'validation' as it's the standard name in Hugging Face
    'test': test_dataset
})

print(herdphobia_dataset)

# 3. Model Selection and Fine-tuning Preparation

Let's select an appropriate Afrocentric PLM for Hausa language:

In [ ]:
# Choose an Afrocentric PLM that supports Hausa
# Choose an Afrocentric PLM that supports Hausa
model_name = "masakhane/afroxml-r"  # AfroXLMR
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Tokenize all splits
tokenized_datasets = herdphobia_dataset.map(tokenize_function, batched=True)

# Prepare for training
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Load model with 3 output classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 4. Fine-tuning the Model

## 4.1  Updated Metrics Function for 3-Class Classification

In [ ]:
# Define metrics function for multi-class classification
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Multi-class metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    
    # Detailed classification report
    report = classification_report(labels, predictions, output_dict=True)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'class_0_f1': report['0']['f1-score'],  # HT class
        'class_1_f1': report['1']['f1-score'],  # NHT class
        'class_2_f1': report['2']['f1-score']   # IND class
    }

# 5. Training and Evaluation

Now let's leverage the dev set for validation during training:

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate on test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test set results: {test_results}")

# 6. Performance Improvement Strategies for Multi-Class Classification

## 6.1 Handling Class Imbalance

In [ ]:
# Check if there's class imbalance
train_label_counts = train_df['label'].value_counts()
print("Label distribution:", train_label_counts)

# Calculate class weights if needed

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
print("Class weights:", class_weights)

# Apply class weights in the model
# Note: You would typically do this inside the model's forward pass
# But with Hugging Face, we need a custom training loop or a custom model
# For simplicity, we'll use weighted data augmentation instead

# Weighted data augmentation for minority classes
def create_balanced_augmented_dataset(df, class_counts):
    # Find the majority class count
    max_count = class_counts.max()
    
    # Create dataframes for each class
    class_dfs = {}
    for class_label in class_counts.index:
        class_dfs[class_label] = df[df['label'] == class_label]
    
    # Augment each class to reach similar size as majority class
    augmented_dfs = []
    for class_label, class_df in class_dfs.items():
        if len(class_df) < max_count:
            # Calculate how many times to duplicate
            augment_factor = int(np.ceil(max_count / len(class_df))) - 1
            
            # Apply augmentation
            augmented_class_df = class_df.copy()
            for _ in range(augment_factor):
                # Apply text augmentation to the minority class
                augmented_texts = []
                augmented_labels = []
                
                for _, row in class_df.iterrows():
                    # Apply simple augmentation (you can use the techniques defined earlier)
                    # For now, just duplicate the example
                    augmented_texts.append(row['text'])
                    augmented_labels.append(row['label'])
                
                temp_df = pd.DataFrame({
                    'text': augmented_texts,
                    'label': augmented_labels
                })
                
                augmented_class_df = pd.concat([augmented_class_df, temp_df], ignore_index=True)
            
            # Take only what we need to balance
            augmented_class_df = augmented_class_df.sample(max_count, replace=False)
            augmented_dfs.append(augmented_class_df)
        else:
            augmented_dfs.append(class_df)
    
    # Combine all balanced classes
    balanced_df = pd.concat(augmented_dfs, ignore_index=True)
    return balanced_df

# Create a more balanced dataset
balanced_train_df = create_balanced_augmented_dataset(train_df, train_label_counts)
print(f"Original training set size: {len(train_df)}")
print(f"Balanced training set size: {len(balanced_train_df)}")
print("New class distribution:", balanced_train_df['label'].value_counts())

# Convert to Hugging Face dataset
balanced_train_dataset = Dataset.from_pandas(balanced_train_df)
balanced_dataset_dict = DatasetDict({
    'train': balanced_train_dataset,
    'validation': dev_dataset,
    'test': test_dataset
})

# Tokenize the balanced dataset
tokenized_balanced_datasets = balanced_dataset_dict.map(tokenize_function, batched=True)
tokenized_balanced_datasets = tokenized_balanced_datasets.remove_columns(["text"])
tokenized_balanced_datasets = tokenized_balanced_datasets.rename_column("label", "labels")
tokenized_balanced_datasets.set_format("torch")

## 6.2 Advanced Fine-tuning Techniques

In [ ]:
# Set up improved training arguments
improved_training_args = TrainingArguments(
    output_dir="./improved_results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,  # Mixed precision training
    warmup_ratio=0.1,
    push_to_hub=False,
    logging_steps=50,
)

# Initialize improved trainer with balanced dataset
improved_trainer = Trainer(
    model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3),
    args=improved_training_args,
    train_dataset=tokenized_balanced_datasets["train"],
    eval_dataset=tokenized_balanced_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Train improved model
improved_trainer.train()

# Evaluate improved model
improved_test_results = improved_trainer.evaluate(tokenized_balanced_datasets["test"])
print(f"Improved model results: {improved_test_results}")

# Display per-class performance
print("Per-class F1 scores:")
print(f"Hate speech (HT): {improved_test_results['eval_class_0_f1']:.4f}")
print(f"Non-hate speech (NHT): {improved_test_results['eval_class_1_f1']:.4f}")
print(f"Indeterminate (IND): {improved_test_results['eval_class_2_f1']:.4f}")

## 6.3 Confusion Matrix Analysis

In [ ]:
# Get predictions from the model
test_predictions = improved_trainer.predict(tokenized_balanced_datasets["test"])
predicted_labels = np.argmax(test_predictions.predictions, axis=1)
true_labels = tokenized_balanced_datasets["test"]["labels"]

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
class_names = ["HT", "NHT", "IND"]  # Replace with actual class names

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.close()

# Calculate per-class metrics
print("Detailed classification report:")

print(classification_report(true_labels, predicted_labels, target_names=class_names))

## 6.4 Error Analysis and Model Interpretation

In [ ]:
# Get examples where the model made mistakes
def get_error_examples(test_dataset, true_labels, predicted_labels, n=10):
    error_indices = np.where(true_labels != predicted_labels)[0]
    
    if len(error_indices) == 0:
        print("No errors found!")
        return
    
    # Select a random sample of errors (up to n)
    sample_size = min(n, len(error_indices))
    selected_indices = np.random.choice(error_indices, size=sample_size, replace=False)
    
    # Get the original text and labels
    error_examples = []
    for idx in selected_indices:
        example = {
            'text': test_df.iloc[idx]['text'],
            'true_label': class_names[true_labels[idx]],
            'predicted_label': class_names[predicted_labels[idx]]
        }
        error_examples.append(example)
    
    return error_examples

error_examples = get_error_examples(test_df, true_labels, predicted_labels)

print("Error Analysis:")
for i, example in enumerate(error_examples):
    print(f"Example {i+1}:")
    print(f"Text: {example['text']}")
    print(f"True label: {example['true_label']}")
    print(f"Predicted label: {example['predicted_label']}")
    print("---")

# 7. Publishing to Hugging Face Hub

Let's publish our best model to the Hugging Face Hub

In [ ]:
# Log in to Hugging Face
notebook_login()

# Choose the best model
best_trainer = improved_trainer

# Create a model card
model_card = """
---
language: hau
license: mit
datasets:
  - hausanlp/HERDPhobia
tags:
  - hate-speech-detection
  - hausa
  - african-nlp
  - fulani-herdsmen
---

# HERDPhobia Hate Speech Detection Model

This model is fine-tuned on the HERDPhobia dataset for detecting hate speech against Fulani herdsmen in Nigeria in Hausa language text.

## Model Description

This model is based on [masakhane/afroxml-r](https://huggingface.co/masakhane/afroxml-r) fine-tuned on the HERDPhobia dataset. It performs 3-class classification:
- Hate speech (HT)
- Non-hate speech (NHT)
- Indeterminate (IND)

## Intended Use

This model is intended for detecting hate speech against Fulani herdsmen in Hausa language text from social media and other online sources.

## Training Data

The model was trained on the HERDPhobia dataset, which contains annotated tweets in Hausa.

## Performance

The model achieves the following scores on the test set:
- Overall Accuracy: {improved_test_results['eval_accuracy']:.4f}
- Overall F1 Score: {improved_test_results['eval_f1']:.4f}
- Per-class F1 scores:
  - Hate speech (HT): {improved_test_results['eval_class_0_f1']:.4f}
  - Non-hate speech (NHT): {improved_test_results['eval_class_1_f1']:.4f}
  - Indeterminate (IND): {improved_test_results['eval_class_2_f1']:.4f}

## Improvement Strategies

1. Class balancing techniques were applied to handle uneven class distribution
2. Hyperparameter optimization improved overall performance
3. Error analysis was conducted to understand model limitations

## Limitations

This model may have limitations in detecting subtle forms of hate speech or hate speech expressed in dialects or slang not well-represented in the training data.
"""

# Save the model card
with open("README.md", "w") as f:
    f.write(model_card)

# Push to Hugging Face Hub
model_name_on_hub = "YOUR_USERNAME/hausa-herdphobia-classifier"  # Replace with your username

best_trainer.push_to_hub(
    repo_id=model_name_on_hub,
    commit_message="Add fine-tuned model for Hausa hate speech detection against Fulani herdsmen"
)

print(f"Model successfully pushed to {model_name_on_hub}")

# 8. Conclusion and Analysis Report